# Extreme Graident Boosting

GBM'in hız ve tahmin performamsını arttırmak üzere optimize edilmiş,

ölçeklenebilir ve farkşlı platformlara uyarlanabilir halidir

lightGBM ve catGBM 'e kadar şüpheye mi düştün XGBoost kullan denilecek kadar iyi performansı vardır ancak light gbm bu algoritmayı sollamıştır

# Model

In [26]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor

from warnings import filterwarnings
filterwarnings('ignore')

In [27]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [28]:
import xgboost as xgb

In [29]:
# Geliştiricisinin uyarısına göre XGBoost'a pandas data frame yerine xgb nin kendi veri yapısını kullanırsak daha iyi performans
#elde ederiz bu yüzden değişkenlerde dönüşüm yapacağız

In [30]:
DM_train=xgb.DMatrix(data=X_train,label=y_train)
DM_test=xgb.DMatrix(data=X_test,label=y_test)

In [31]:
from xgboost import XGBRegressor

In [32]:
xgb_model=XGBRegressor().fit(X_train,y_train)

# Tahmin

In [33]:
y_pred=xgb_model.predict(X_test)

In [34]:
np.sqrt(mean_squared_error(y_test,y_pred))

355.46515176059927

# Model Tuning

In [35]:
params={"colsample_bytree":[0.4,0.5,0.6,0.9,1],#ağaç başına değişkenlerden alıcak örnek
        "n_estimators":[100,200,500,1000],#ağaç sayısı
        "max_depth":[2,3,4,5,6],#maximum derinlik. Overfitting ile mücedele için var. Karmaşıklık parametresi olarakta adlandırılabilir
        "learning_rate":[0.1,0.01,0.5]#daraltm adım boyu. OverFitting ile mücedale için var değeri 0-1 arasında olması beklenir küçük olması yavaşlatır ama daha iyisonuç veriri
        }

In [36]:
xgb_cv=GridSearchCV(xgb_model,
                    param_grid=params,
                    cv=10,
                    n_jobs=-1,
                    verbose=2).fit(X_train,y_train)

Fitting 10 folds for each of 300 candidates, totalling 3000 fits
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=2, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=2, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=2, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=2, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=2, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=2, n_estimators=200; total time=   0.1s
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=2, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=2, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=2, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=0.4, lear

In [37]:
xgb_cv.best_params_

{'colsample_bytree': 0.5,
 'learning_rate': 0.1,
 'max_depth': 2,
 'n_estimators': 1000}

In [38]:
xgb_final=XGBRegressor(colsample_bytree=0.5,learning_rate=0.1,max_depth=2,n_estimators=1000).fit(X_train,y_train)
y_pred=xgb_final.predict(X_test)
np.sqrt(mean_squared_error(y_pred,y_test))

357.4251457287713